In [3]:
# Imports

from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import os

In [4]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [5]:
openai = OpenAI()
OPENAI_MODEL = "gpt-4o-mini"

In [22]:
system_prompt = """
You are a highly skilled assistant specialized in generating Python unit tests. Your task is to create valid and well-structured unit tests for a given Python function. Output only the test class and test methods directly, with no introduction, explanation, or summary text.
If no Python function is provided in the input, respond with the following message: 'Please provide a Python function so that I can generate unit tests for it.'

When generating tests:
- Use the unittest framework.
- Ensure the test class and methods are formatted as valid Python code.
- Include specific test cases for edge cases, typical use cases, and invalid inputs.
- Ensure appropriate assertions for each test case.
- Mock external dependencies if necessary.

The output must strictly follow this format:

'class Test[FunctionName](unittest.TestCase):
    
    def test_case_name(self):
        # Comment describing the purpose of this test case
        self.assertEqual(actual, expected)
'

Do not include any additional text, explanations, or summaries. Only provide the Python test code as a unittest class or request a function if none is given.
"""

In [7]:
def user_prompt_for(python):
    user_prompt = "Below is a Python function for which I need unit tests. Include edge cases, typical use cases, and invalid inputs."
    user_prompt += "Please analyze the function and generate comprehensive test cases using the `unittest` framework."
    user_prompt += "If necessary, use mocking for external dependencies.\n\n"
    user_prompt += python
    return user_prompt

In [11]:
def messages_for_python(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [20]:
def documentation_python(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for_python(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python\n','').replace('import unittest\n','').replace('```','')

In [13]:
simple_function_python = """
def calculate_area(length, width):
    return length * width
"""

In [14]:
documentation_python(simple_function_python)

<generator object documentation_python at 0x0000024A6F9A8040>

In [25]:
with gr.Blocks() as ui:
    gr.Markdown("# Python Unit Test Generator")
    gr.Markdown("Paste your Python function in the left panel, and click the button to 'Generate Unit Tests'.")
    
    with gr.Row():
        code_input = gr.Code(label="Code snippets:", lines=10, value=simple_function_python, language="python")
        code_doc = gr.Code(label="Unit tests:", language="python")
    with gr.Row():
        convert = gr.Button("Generate Unit Tests")
    
    convert.click(documentation_python, inputs=[code_input], outputs=[code_doc])
    
ui.launch(inbrowser=True, share=True)

* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://fb3f65e344357b4819.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
